In [ ]:
#from ... import preprocessor

In [3]:
import pandas as pd
import geopandas as gpd

# Importing and preprocessing clean data

In [4]:
clean_data = gpd.read_file("../raw_data/project_data_clean.shp")

In [8]:

X = clean_data.drop(columns=['green_roof', 'geometry'])
y = clean_data['green_roof']

,usetype_bl,district,built_type,residents,air_pollut,thermal_st,social_sta,social_dyn,rent,unemp_bene,social_hou,hous_assoc,rent_durat,aparts_sol,subsidized
0,Verwaltung,Charlottenburg-Wilmersdorf,Gemeinbedarfs- und Sondernutzung,NaN,hoch,mittel,mittel,positiv,12.90,0.0787,0.0314,0.0393,0.6566,70.400002,1
1,Verwaltung,Charlottenburg-Wilmersdorf,Gemeinbedarfs- und Sondernutzung,NaN,hoch,mittel,mittel,stabil,12.90,0.0787,0.0314,0.0393,0.6566,70.400002,1
2,Freistehende Einfamilienhäuser mit Gärten,Charlottenburg-Wilmersdorf,Wohnnutzung,128.0,mittel,hoch,mittel,stabil,12.90,0.0787,0.0314,0.0393,0.6566,70.400002,1
3,Verwaltung,Charlottenburg-Wilmersdorf,Gemeinbedarfs- und Sondernutzung,NaN,mittel,hoch,mittel,stabil,12.90,0.0787,0.0314,0.0393,0.6566,70.400002,1
4,Freistehende Einfamilienhäuser mit Gärten,Charlottenburg-Wilmersdorf,Wohnnutzung,115.0,mittel,hoch,mittel,stabil,12.90,0.0787,0.0314,0.0393,0.6566,70.400002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18010,Gleiskörper,Marzahn-Hellersdorf,Verkehrsfläche (ohne Straßen),NaN,mittel,hoch,mittel,negativ,8.62,0.2031,0.0255,0.3833,0.6545,6.070000,1
18011,Gleiskörper,Marzahn-Hellersdorf,Verkehrsfläche (ohne Straßen),NaN,mittel,hoch,mittel,negativ,8.62,0.2031,0.0255,0.3833,0.6545,6.070000,1
18012,Bahnhof und Bahnanlagen ohne Gleiskörper,Marzahn-Hellersdorf,Verkehrsfläche (ohne Straßen),NaN,mittel,hoch,mittel,negativ,8.62,0.2031,0.0255,0.3833,0.6545,6.070000,1
18013,Gleiskörper,Marzahn-Hellersdorf,Verkehrsfläche (ohne Straßen),NaN,mittel,hoch,mittel,negativ,8.62,0.2031,0.0255,0.3833,0.6545,6.070000,1


# Model training and export

# Visualization